# 1. Load Packages

In [ ]:
from gandalf_doe.experiment import Experiment
from gandalf_doe.domain import Domain, Variable
import numpy as np
import pandas as pd

# 2. Creating the design space

### 2.1 Create a domain by calling the class

In [ ]:
domain = Domain()

### 2.2 Add variables to the domain with `add_variable`

In [ ]:
first_variable = Variable(name="Temperature",   # Name of the variable
                          desc="reaction_temperature",  # A description of the variable
                          domain=np.arange(523, 783, 10),  # The possible values of the 
                          var_type="discrete",  # Variable type: continuous, discrete, or categorical
                         )

In [ ]:
domain.add_variable([first_variable])

In [ ]:
# Add more variables

domain.add_variable([Variable("Pressure", "reactor_pressure", np.arange(1, 11, 1), "discrete")])
domain.add_variable([Variable("GHSV", "gas_hourly_space_velocity", np.arange(3300, 28050, 1650), "discrete")])
domain.add_variable([Variable("Ni", "nickel_load", np.arange(0 , 26, 1), "discrete")])
domain.add_variable([Variable("Co", "cobalt_load", np.arange(0 , 11, 1), "discrete")])
domain.add_variable([Variable("Calcination", "calcination_temperature", np.arange(623 , 973, 50), "discrete")])
domain.add_variable([Variable("Reduction", "reduction_temperature", np.arange(623 , 973, 50), "discrete")])

### 2.3 Set up the design space

In [ ]:
domain.setup_space()

# 3. Initialize the active learning process

### 3.1 Create the experiment

In [ ]:
exp = Experiment(domain=domain,  # The design space
                 pool_size=100000,  # The size of the initial pool
                 n_init=3,  # Number of initial data points that should be returned
                 normalize=True,  # Normalizing the inputs
                 mode="EMOC",  # Acquisition function
                 clustering=True,  # Use clustering in EMOC as described in paper
                 scaling=200,  # Scaling factor for pool after each iteration
                )

### 3.2 Suggest initial experiments

In [ ]:
initial_suggestions = exp.initialize_experiments()

In [ ]:
initial_suggestions

# 4. Suggest one new experiment with EMOC

In [ ]:
dataset = pd.read_excel("../test_data/dataset.xlsx", index_col=None)  # Suggest new data points given previous data

In [ ]:
new_suggestion = exp.suggest_experiments(previous=dataset.copy())
new_suggestion.tail(5)

# 5. Suggest a batch of experiments

In [ ]:
dataset = pd.read_excel("../test_data/dataset.xlsx", index_col=None)

In [ ]:
new_suggestions = exp.suggest_batch_of_experiments(previous=dataset.copy(),  # previous data
                                                   n_clusters=3,   # number of different clusters to be picked
                                                   n_conditions=3,   # number of conditions similar to the value picked per cluster
                                                   constants=[3, 4, 5, 6],  # which parameters to be kept constant when picking conditions
                                                  )
new_suggestions.tail(20)

# 6. Predict outcome

In [ ]:
predictions = exp.predict_outcome(x_new=np.array([663.0, 2.0, 12500.0, 20.0, 3.0, 723.0, 723.0]), 
                                  previous=dataset)

In [ ]:
predictions